# MK_Analysis
- This jupyter notebook is designed to analyze proplatelet production of Megakaryocytes from tiff time-lapses taken by the Incucyte FLR (10X, 1280x1024). 

#### The Analysis workflow occurs in 2 steps:

1 ilastikProcessing - Unpacks tiff stacks & generates binary probabilities from phase images through the ilastik project (.ilp) file.


2 Production Pipeline - Primary pipeline for proplatelet production analysis. Important output files are listed as follows:
    
    a. 'results_Image.csv' -> meg & proplatelet count
    
    b. 'results_cell/pplt.properties' -> use in CPA to train classifiers
    
    c. 'labels' of proplatelet objects used for the Skeleton pipeline
    
    d. 'overlay' of phase images, labeling megs as red & pplts as green

## ilastikProcessing
To effectively use ilastik some formatting must be done before and after ilastik processes the images. The input is assumed to be a time-series of images stored in a multi-page TIFF.

In [1]:
%matplotlib inline
import glob
import h5py
import matplotlib
import numpy
import os
import os.path
import pandas
import re
from shutil import copy2
import skimage
import skimage.exposure
import skimage.io
import subprocess
# from tkinter.filedialog import askdirectory
from tqdm._tqdm_notebook import tqdm_notebook

C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# #Unpack (or separate) input images into single files.

### Update input image variables
Update the variable *image_directory* with the path to a folder that contains the input images. Update the *regex_image* variable to process only the images that match the regular expression. If the *regex_image* variable is equal to `(.*)\.tif`, then any *.tif* in the folder will be processed.

If a filename matches the *regex_single* regular expression, then it is assumed that this image has already been unpacked. An unpacked single image will have timepoint appended to the end of the file following the pattern `\-\d{4}\.tif`

*path_to_ilastik* is a string with the path to the ilastik software for [running headless](http://ilastik.org/documentation/basics/headless.html).

### **EDIT DIRECTORY BELOW**

In [2]:
image_directory = r"C:\Users\Prakrith\Desktop\180221_HCS_Bort"
# imdir_ilastik = r"C:\Users\Prakrith\Desktop\180312_ARW\ilastik"
# imdir_single = r"C:\Users\Prakrith\Desktop\180312_ARW\single_images"
# output_directory = r"C:\Users\Prakrith\Desktop\180312_ARW\output"

In [3]:
path_to_ilastik = r"C:\Program Files\ilastik-1.3.0b4\run-ilastik.bat"
path_to_project = r"C:\Users\Prakrith\Documents\GitHub\Test\ilps\180331_MK_384.ilp"
path_to_cellprofiler = r"C:\Program Files (x86)\CellProfiler\CellProfiler.exe"
path_to_cp_pipeline = r"C:\Users\Prakrith\Documents\GitHub\Test\pipelines\180325_MP.cppipe"
regex_image = "(.*)\.tif" #stack
regex_single = ".*\-\d{4}\.tif" #slice
re_image = re.compile(regex_image)
re_single = re.compile(regex_single)

In [4]:
os.makedirs(os.path.join(image_directory, "single_images"), exist_ok=True) 
imdir_single = os.path.join(image_directory,"single_images")
os.makedirs(os.path.join(image_directory, "ilastik"), exist_ok=True)
imdir_ilastik = os.path.join(image_directory,"ilastik")
os.makedirs(os.path.join(image_directory, "output"), exist_ok=True)
output_directory = os.path.join(image_directory,"output")
# os.makedirs(os.path.join(image_directory, "skeleton")), exist_ok=True)
# skeleton_directory = os.path.join(image_directory,"skeleton")

## Methods to import image metadata
* *is_my_file* will use the regular expression to filter image files to be processed.
* *make_dict* parses a file to be processed and places metadata into a dictionary.

Parse the files to be processed and then place the metadata into a Pandas dataframe.

In [5]:
def is_my_file(filename, re_image, re_single):
    
    mybool = False
    
    if (    re_image.match(filename) != None 
        and re_single.match(filename) == None
       ):
        
        mybool = True
        
    return mybool


def make_dict(filename, path, re_obj):
    
    my_dict = re_obj.match(filename).groupdict()
    
    my_dict["filename"] = filename
    
    my_dict["path"] = path
    
    return my_dict

In [6]:
image_files_dict = [make_dict(f, image_directory, re_image) for f in os.listdir(image_directory) if is_my_file(f, re_image, re_single)]
image_df = pandas.DataFrame(image_files_dict)

## Unpack the multi-page TIFF images
For every image, create a single image file for each timepoint. The input images are assumed to be RGB, which has 3 dimensions (length, width, color). The multipage TIFF of RGB images will have 4 dimensions (timepoints, length, width, color). 

*If the upstream workflow changes and the input image format is altered, then the conditional logic below will need to be updated, specifically the logic based on the shape of the input images.*

### Are the images across experiments similar enough to treat equally
One concern is that overfitting from training a classification model either through ilastik or CellProfiler analyst. The training set needs to be representative of the possibility space. This is accomplished by choosing a large enough image set that includes images of all states of interest including undifferentiated and fully differentiated megakaryocytes.

We also want to eliminate noise from known sources of variablity that could potentially weaken the classifier. The primary sources of noise in the images will be non-uniform illumination and differences in exposure. Non-uniform illumination is difficult to correct, because the background is actually in the middle of the intensity range and the signal occupies both high and low intensities.



In [7]:
#filelist = glob.glob("D:\Prakrith\MK_Differentiation_Kyle\images\single_images\*.tif")

#for f in filelist:
    
#    im = skimage.io.imread(f)
    
#    im2 = skimage.color.rgb2gray(im)
        
#    im2 = skimage.img_as_ubyte(im2)

#    skimage.io.imsave(f, im2)    DOWNSAMPLE

In [8]:
def df_stack_image(p):
    
    im = skimage.io.imread(os.path.join(p["path"], p["filename"]))
    
    if len(im.shape) < 4:
        
        retest = re_image.match(p["filename"])

        retest.group(1)

        fname = "{0}-{1:04d}.tif".format(retest.group(1), 0)
        
        im2 = skimage.color.rgb2gray(im)
        
        im2 = skimage.img_as_ubyte(im2)

        skimage.io.imsave(os.path.join(p["path"], fname), im2)
        
    else:
    
        number_of_timepoints = im.shape[0]

        for i in range(number_of_timepoints):

            retest = re_image.match(p["filename"])

            retest.group(1)

            fname = "{0}-{1:04d}.tif".format(retest.group(1), i)
            
            im2 = skimage.color.rgb2gray(im[i,:,:,:])
        
            im2 = skimage.img_as_ubyte(im2)

            skimage.io.imsave(os.path.join(p["path"], "single_images", fname), im2)

In [9]:
if image_df.empty is False:
    
    # Note that this can fail if the input images aren't in the expected format
    # If you receive an error, double check the format of the input images, e.g. are they RGB?
    tqdm_notebook.pandas(desc="unpack")
    _ = image_df.progress_apply(df_stack_image, axis=1)

else:
    
    print("no images to unpack")

HBox(children=(IntProgress(value=0, description='unpack', max=78), HTML(value='')))

C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


# Run ilastik

Using the single images created earlier, process the images using ilastik. First, create another dataframe with the single image metadata. Note, this has been written for running on Windows.

## Process ilastik output for CellProfiler
ilastik will output and HDF5 file that must be parsed for use as input to CellProfiler. This workflow assumes the default export settings are being used in ilastik. We have observed performance costs when changing the exporting settings to formats beyond the standard ilastik HDF5 file. For example, exporting TIFF images changes the shape of the exported data from yxc (the default) to cyx. This rearrangement will cause downstream errors, because the code as written expects the channel to be the third dimension.

### ilastik stage-1 labels
The project file *Mouse_MK_Training_180101_v1.ilp* has the following labels that are stored in the same order within the HDF5 output.
1. background
2. border_white
3. cell
4. protrusion
5. background_border
6. not_cell

### ilastik stage-2 labels
The probability images output from the second-stage classification are stored in the ilastik folder as pngs, in the following order:
0. background
1. cell_boundary
2. cell
3. protrusion

In [10]:
def is_my_file(filename, re_obj):
    
    mybool = False
    
    if re_obj.match(filename) != None:
        
        mybool = True
        
    return mybool


def make_dict(filename, path, re_obj):
    
    my_dict = re_obj.match(filename).groupdict()
    
    my_dict["filename"] = filename
    
    my_dict["path"] = path
    
    return my_dict

In [11]:
image_files_dict = [make_dict(f, imdir_single, re_single) for f in os.listdir(imdir_single) if is_my_file(f, re_single)]
image_df = pandas.DataFrame(image_files_dict)

In [12]:
def df_ilastik(p):
    
    filename = os.path.join(p["path"], p["filename"])
    
    filename_noext = os.path.splitext(p["filename"])[0]
    
    filename_h5 = "{}_Probabilities Stage 2.h5".format(filename_noext)
    
    # Run ilastik using subprocess
    
    process = subprocess.Popen([path_to_ilastik, 
                  "--headless",
                  "--export_source=probabilities stage 2",
                  "--output_format=hdf5",
                  r"--project={}".format(path_to_project),
                  filename
                 ], stdout=subprocess.PIPE)
    
    out, err = process.communicate()
    
    # unpack the HDF5 file
    
    label_list = ["background", "protrusion", "cell_boundary", "cell"]
    
    path_h5 = os.path.join(p["path"], filename_h5)
    
    with h5py.File(path_h5, "r") as ilastik_hdf5:
    
        ilastik_probabilities = ilastik_hdf5["exported_data"].value
    
        for i in range(ilastik_probabilities.shape[2]):
            im = skimage.img_as_uint(ilastik_probabilities[:, :, i])
        
            filename_slice = "{}_{}_prbstg2_{}.png".format(filename_noext, label_list[i], i)
        
            skimage.io.imsave(os.path.join(p["path"], "..", "ilastik", filename_slice), im)
    
    os.remove(path_h5)

In [13]:
tqdm_notebook.pandas(desc="run ilastik")
_ = image_df.progress_apply(df_ilastik, axis=1)

HBox(children=(IntProgress(value=0, description='run ilastik', max=1872), HTML(value='')))

C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:\Users\Prakrith\Desktop\180221_HCS_Bort\single_images\..\ilastik\180219_HCS_Ctrl_A21_1-0012_cell_boundary_prbstg2_2.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:\Users\Prakrith\Desktop\180221_HCS_Bort\single_images\..\ilastik\180219_HCS_Ctrl_A21_1-0013_cell_boundary_prbstg2_2.png is a low contrast image
  warn('%s is a low contrast image' % fname)


# Run CellProfiler

## Make a filelist
Add the paths to each file that will be processed by CellProfiler into a text file.

## Run CellProfiler Pipeline
Use subprocess to run CellProfiler on the images to be processed.

Note, that a model that filters protrusions was trained in CellProfiler Analyst outside of this workflow. The model has to be in the input folder to be found by CellProfiler. ("CPTemp_in" folder on Desktop, fgb_rules_pplt.txt)

In [14]:
CPA_Rules = r'C:\Users\Prakrith\Desktop\CPTemp_in\fgb_rules_pplt.txt'
copy2(CPA_Rules, image_directory)

'C:\\Users\\Prakrith\\Desktop\\180221_HCS_Bort\\fgb_rules_pplt.txt'

In [15]:
single_list = glob.glob(os.path.join(imdir_single,"*.tif"))
ilastik_list = glob.glob(os.path.join(imdir_ilastik,"*.png"))
big_list = single_list + ilastik_list
with open(os.path.join(image_directory,"filelist.txt"), 'w') as f:
    for item in big_list:
        f.write("{}\n".format(item))

In [16]:
process = subprocess.Popen([path_to_cellprofiler,
                  "--run-headless",
                  "--pipeline={}".format(path_to_cp_pipeline),
                  "--file-list={}".format(os.path.join(image_directory,"filelist.txt")),
                  "--image-directory={}".format(image_directory),
                  "--output-directory={}".format(output_directory)
                 ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

out, err = process.communicate()

## Quantify Proplatelet Production
From the csvs generated by CellProfiler, the file 'results_Image' is parsed, & proplatelet production is quantified into Raw_Pct.csv. Raw_Pct is formatted into Graph_Pct.csv for graphing ease.

In [17]:
result = open(os.path.join(output_directory,r'results_Image.csv'));
header = ["URL_phase","Count_proplatelets","Count_megs","ImageNumber"]; 
df = pandas.read_csv(result, usecols = header, index_col = False);

In [18]:
def f(x):
    return (((x[1])/(x[0] + x[1]))*100)

In [19]:
df.to_csv(os.path.join(output_directory,r'Raw_Pct.csv'));

In [20]:
df

,Count_megs,Count_proplatelets,ImageNumber,URL_phase
0,110.0,24.0,1,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
1,95.0,27.0,2,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
2,85.0,36.0,3,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
3,81.0,34.0,4,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
4,68.0,34.0,5,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
5,65.0,26.0,6,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
6,70.0,28.0,7,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
7,59.0,31.0,8,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
8,57.0,25.0,9,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...
9,54.0,25.0,10,file:///C:/Users/Prakrith/Desktop/180221_HCS_B...


In [21]:
x = df.apply(f,axis=1);

In [22]:
x = df.apply(f,axis=1);
df['normalized_pct'] = x;
df = df.set_index("ImageNumber");
df.to_csv(os.path.join(output_directory,r'Raw_Pct.csv'));
df2 = pandas.DataFrame();

In [23]:
n = 24; #change n to equivalent timepoints

In [27]:
for i in range(0,len(x),n):
    slc = x.iloc[i:i+n];
    slc = slc.reset_index(drop=True);
    df2 = pandas.concat([df2,slc],axis=1,ignore_index=True);

In [28]:
df2.to_csv(os.path.join(output_directory,r'Graph_Pct.csv'));